# 텐서 다루기

In [15]:
# 라이브러리
from torch import torch

In [16]:
# 텐서생성
data = [[1,2],[3,4]]
print(torch.tensor(data))
print(torch.tensor(data, device="cpu"))
print(torch.tensor(data, dtype=torch.float64))

tensor([[1, 2],
        [3, 4]])
tensor([[1, 2],
        [3, 4]])
tensor([[1., 2.],
        [3., 4.]], dtype=torch.float64)


In [17]:
# ndarray로 변환
temp = torch.tensor(data)
print(temp.numpy())
print(temp.to("cpu").numpy())

[[1 2]
 [3 4]]
[[1 2]
 [3 4]]


In [18]:
# 인덱스 조작
temp = torch.FloatTensor([1, 2, 3, 4, 5, 6, 7])
print(temp[0], temp[1], temp[-1])
print(temp[2:5], temp[4:-1])

tensor(1.) tensor(2.) tensor(7.)
tensor([3., 4., 5.]) tensor([5., 6.])


In [20]:
# 텐서 연산
v = torch.tensor([1, 2, 3])
w = torch.tensor([3, 4, 6])
print(w - v)

tensor([2, 2, 3])


In [21]:
# 텐서 차원 조작
temp = torch.tensor([
    [1, 2], [3, 4]
])
print(temp.shape)
print(temp.view(4, 1))
print(temp.view(-1))
print(temp.view(1, -1))
print(temp.view(-1, 1))

torch.Size([2, 2])
tensor([[1],
        [2],
        [3],
        [4]])
tensor([1, 2, 3, 4])
tensor([[1, 2, 3, 4]])
tensor([[1],
        [2],
        [3],
        [4]])


# 커스텀 데이터셋 구현

In [22]:
import pandas as pd
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, csv_file):
        self.label = pd.read_csv(csv_file)
    
    def __len__(self):
        return len(self.label)
    
    def __getitem__(self, idx):
        sample = torch.tensor(self.label.iloc[idx,0:3]).int()
        label = torch.tensor(self.label.iloc[idx,3]).int()
        return sample, label
    
tensor_dataset = CustomDataset('./test.csv')
dataset = DataLoader(tensor_dataset, batch_size=4, shuffle=True)

# 모델 정의

### nn.Module() 상속

In [23]:
import torch.nn as nn

class MLP(nn.Module):
    def __init__(self, inputs):
        super(MLP, self).__init__()
        self.layer = nn.Linear(inputs, 1)
        self.activation = nn.Sigmoid()

    def forward(self, X):
        X = self.layer(X)
        X = self.activation(X)
        return X

### Sequential 신경망을 정의

In [26]:
import torch.nn as nn

class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=5),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2)
        )

        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=30, kernel_size=5),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2)
        )

        self.layer3 = nn.Sequential(
            nn.Linear(in_features=30*5*5, out_features=10, bias=True),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = x.view(x.shape[0], -1)
        x = self.layer3(x)
        return x
    
model = MLP()

print(list(model.children()))
print(list(model.modules()))

[Sequential(
  (0): Conv2d(3, 64, kernel_size=(5, 5), stride=(1, 1))
  (1): ReLU(inplace=True)
  (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
), Sequential(
  (0): Conv2d(64, 30, kernel_size=(5, 5), stride=(1, 1))
  (1): ReLU(inplace=True)
  (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
), Sequential(
  (0): Linear(in_features=750, out_features=10, bias=True)
  (1): ReLU(inplace=True)
)]
[MLP(
  (layer1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(64, 30, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Sequential(
    (0): Linear(in_features=750, out_features=10, bias=True)
    (1): ReLU(inplace=True)
  )
), Sequential(
  (0)

### 함수로 신경망을 정의

In [ ]:
def MLP(in_features=1, hidden_features=20, out_features=1):
    hidden = nn.Linear(in_features=in_features, out_features=hidden_features, bias=True)
    activation = nn.ReLU
    output = nn.Linear(in_features=hidden_features, out_features=out_features, bias=True)
    net = nn.Sequential(hidden, activation, output)
    return net

# 파라미터 정의

In [ ]:
from torch.optim import optimizer

criterion = torch.nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer=optimizer, lr_lambda=lambda epoch: 0.95 ** epoch)

for epoch in range(1, 100+1):
    for x, y in dataloader:
        optimizer.zero_grad()
loss_fn(model(x), y).backward()
optimizer.step()
scheduler.step()

# 모델 훈련

In [ ]:
for epoch in range(100):
    yhat = model(x_train)
    loss = criterion(yhat, y_train)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

# 모델 평가

In [56]:
import torch
import torchmetrics
metric = torchmetrics.Accuracy(task='multiclass', num_classes=5)

n_batches = 10
for i in range(n_batches):
    preds = torch.randn(10, 5).softmax(dim=-1)
    target = torch.randint(5, (10,))

    acc = metric(preds, target)
    print(f"Accuracy on batch {i}: {acc}")

acc = metric.compute()
print(f"Accuracy on all data: {acc}")

Accuracy on batch 0: 0.4000000059604645
Accuracy on batch 1: 0.30000001192092896
Accuracy on batch 2: 0.20000000298023224
Accuracy on batch 3: 0.20000000298023224
Accuracy on batch 4: 0.30000001192092896
Accuracy on batch 5: 0.0
Accuracy on batch 6: 0.30000001192092896
Accuracy on batch 7: 0.20000000298023224
Accuracy on batch 8: 0.4000000059604645
Accuracy on batch 9: 0.0
Accuracy on all data: 0.23000000417232513


# 모니터링

In [ ]:
import torch
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter("./")

for epoch in range(num_epochs):
    model.train()
    batch_loss = 0.0

    for i, (x, y) in enumerate(dataloader):
        x, y = x.to(device).float(), y.to(device).float()
        outputs = model(x)
        loss = criterion(outputs, y)
        writer.add_scalar("Loss", loss, epoch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

writer.close()

'''
터미널에
tensorboard --logdir=./tensorboard --port=6006
'''